# 🧬 **AI for Biology** 🧬

<a href="https://ibb.co/Cs0GsQD"><img src="https://i.ibb.co/mFzWF4g/d3ccc3f8-69e2-428f-8ec4-896221936735.webp" alt="Scifi collage of AI in biology" border="0"></a>

<a href="https://colab.research.google.com/github/deep-learning-indaba/indaba-pracs-2024/blob/main/practicals/AI_for_Biology/AI_for_Biology_French.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

© Deep Learning Indaba 2024. Apache License 2.0.

**Auteurs :** Natasha Latysheva

**Sujets :** Biologie, ADN, modèles de langage de grande taille, plongements (embeddings) , apprentissage supervisé et auto-supervisé.

**Niveau :** Débutant

**Objectifs :** Comprendre les plongements d'ADN et entraîner un modèle en les utilisant pour résoudre un problème biologique pratique

# AI for Biology
Bienvenue dans le tutoriel pratique **AI for Biology** ! Dans cette session, nous allons :
- Découvrir certains des principaux domaines d'application de l'IA dans les biosciences
- Examiner le rôle de l'ADN et la manière dont les modèles de langage d'ADN sont entraînés
- Extraire et explorer les plongements (embeddings) de l'ADN en utilisant un modèle de langage d'ADN pré-entraîné à la pointe de la technologie
- Se plonger dans un problème pratique de modélisation des séquences d'ADN et de leurs propriétés

**Prérequis :**

1. Connaissances de base en Python
2. Aucune connaissance en biologie requise

**Plan du tutoriel pratique :**

<div align="left">
<a href="https://ibb.co/jryGWdL"><img src="https://i.ibb.co/kS409ph/Screenshot-2024-07-23-at-21-48-22.png" alt="Screenshot-2024-07-23-at-21-48-22" width="400" border="0"></a>
</div>



**Avant de commencer :**

Pour ce tutoriel pratique, vous devrez utiliser un GPU pour accélérer l'entraînement. Pour ce faire, allez dans le menu "Exécution" de Colab, sélectionnez "Changer le type d'exécution", puis dans le menu contextuel, choisissez "GPU" dans la zone "Accélérateur matériel".


Nous pouvons également déjà installer et importer tous les packages requis :
    


In [ ]:
## Installer et importer tout ce qui est requis, télécharger les modèles, télécharger les données.
# @title Installer et importer les packages nécessaires. (Exécuter la cellule)
%%capture

# Installations.
!pip install transformers datasets
!pip install biopython requests h5py
!pip install jax
!pip install flax

# Importations.
import os
import random
import tqdm

import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
import flax.linen as nn
import optax
import torch


import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt
import seaborn as sns

from Bio import Entrez, SeqIO

from sklearn.manifold import TSNE

from transformers import AutoTokenizer, AutoModelForMaskedLM


# Télécharger le modèle de langage d'ADN et le tokenizer.
tokenizer = AutoTokenizer.from_pretrained(
    "InstaDeepAI/nucleotide-transformer-v2-50m-multi-species",
    trust_remote_code=True)

language_model = AutoModelForMaskedLM.from_pretrained(
    "InstaDeepAI/nucleotide-transformer-v2-50m-multi-species",
    trust_remote_code=True)

# Télécharger les plongements pré-extraits pour des chaînes d'ADN aléatoires.
ROOT_DIR = "https://raw.githubusercontent.com/deep-learning-indaba/indaba-pracs-2024/main/practicals/AI_for_Biology/data/"

import pandas as pd
dna_sequences = pd.read_csv(os.path.join(ROOT_DIR, "dna_sequences.csv"))
# (train_df) Données d'entraînement
train_df = pd.read_feather(os.path.join(ROOT_DIR, "train_embeddings.feather"))
# (valid_df) Données de validation
valid_df = pd.read_feather(os.path.join(ROOT_DIR, "valid_embeddings.feather"))


In [ ]:
# @title Vérifier TPU/GPU. (Exécuter la cellule)
import jax
num_devices = jax.device_count()
device_type = jax.devices()[0].device_kind
print(f"Trouvé {num_devices} dispositif(s) JAX de type {device_type}.")

## 1. Applications de l'IA en Biologie

L'IA devient de plus en plus courante dans le domaine biologique et a connu des avancées récentes vraiment passionnantes. Cependant, le domaine en est encore à ses débuts - cela signifie qu'il reste beaucoup de travail intéressant à faire et que c'est un excellent moment pour s'y impliquer !

Voici un rapide aperçu de quelques travaux récents intéressants en IA appliquée à la biologie dans différents domaines.



### Diagnostics médicaux

Les modèles de classification du cancer de la peau tels que [celui du MIT](https://www.science.org/doi/10.1126/scitranslmed.abb3652) atteignent des performances comparables à celles de dermatologues certifiés :
<div align="center">
    <img src="https://wp.technologyreview.com/wp-content/uploads/2021/06/automated-melanoma-detection-small2.gif?w=400" alt="GIF de détection automatisée du mélanome">
</div>


Un [modèle de DeepMind](https://www.nature.com/articles/s41591-018-0107-6) de segmentation et de classification des maladies de la rétine est capable de diagnostiquer de nombreuses affections ophtalmiques à partir de scans rétiniens 3D. Ses performances sont similaires à celles des meilleurs spécialistes de la rétine et surpassent celles de certains experts humains :



<div align="center">
    <img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*cnyoA2T8BFZRBYWnUlYEtQ.gif" alt="GIF de segmentation du scan rétinien">
</div>

[Le modèle SynthSR de Harvard et de l'UCL](https://www.science.org/doi/10.1126/sciadv.add3607) peut prendre des IRM cérébrales cliniques avec n'importe quel contraste, orientation et résolution et les transformer en images 3D haute résolution :

<div align="center">
    <img src="https://www.science.org/cms/10.1126/sciadv.add3607/asset/7a6c5ed9-af95-41d2-b888-2b5a653ea55b/assets/images/large/sciadv.add3607-f1.jpg" alt="Modèle de scan IRM SynthSR" width="400">
</div>


### Pharmacie et développement de médicaments

[Exscientia](https://www.exscientia.com/) a développé le premier médicament conçu par l'IA à entrer dans des essais cliniques (DSP-1181, destiné au traitement du trouble obsessionnel-compulsif).

De nombreux efforts de découverte de médicaments assistés par l'IA utilisent des modèles pour prédire la force avec laquelle les petites molécules se lieront à différentes régions d'une protéine cible impliquée dans une maladie donnée :

<div align="center">
    <img src="https://developer-blogs.nvidia.com/wp-content/uploads/2023/03/bionemo_featured.jpeg" alt="representation d'une poche générale" width="400">
</div>


[BenevolentAI](https://www.benevolent.com/about-us/sustainability/covid-19/) a utilisé l'IA pour identifier le Baricitinib, à l'origine un médicament contre l'arthrite, comme traitement potentiel contre la COVID-19 en 48 heures en utilisant son graphique de connaissances.


<div align="center">
    <img src="https://www.benevolent.com/application/files/6616/7458/5885/Corona_Baricitinib.png" alt="BenevolentAI Baricitinib" width="400">
</div>

[Lien vers une vidéo YouTube intitulée « BenevolentAI · AI-Enabled Drug Discovery »](https://www.youtube.com/watch?v=RPBDhogTIT0)

[Recursion Pharmaceuticals](https://www.recursion.com/) est réputée pour son criblage et son optimisation à haut débit, et a développé des modèles avancés d'imagerie cellulaire :

<div align="center">
    <img src="https://miro.medium.com/v2/resize:fit:1400/0*yVLwEtfojWdnMZfA" alt="Recursion" width="800">
</div>


Ils entraînent des modèles pour prédire les pixels manquants dans les images de cellules de la même manière que les grands modèles de langage prédisent les mots manquants ou masqués dans les phrases :

<div align="center">
    <img src="https://blogs.nvidia.com/wp-content/uploads/2024/05/Recursion-Phenom-AI-model-animation.gif" alt="Recursion" width="800">
</div>

Parmi les autres startups qui font de l'apprentissage profond pour la découverte de médicaments, citons Atomwise, insitro, Insilico Medicine, Deep Genomics et Deepcell.

En outre, de grandes sociétés pharmaceutiques comme Illumina, GSK et Genentech ont mis en place des équipes internes d'apprentissage profond et ont développé des modèles influents tels que [SpliceAI](https://www.cell.com/cell/pdf/S0092-8674(18)31629-5.pdf) (un modèle qui comprend l'épissage) et [PrimateAI](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6237276/) (un modèle qui prédit l'effet clinique des mutations dans les protéines), ce qui témoigne de l'intégration croissante de l'apprentissage profond dans les flux de travail en biologie.


### Biologie moléculaire

Le [modèle AlphaFold de DeepMind](https://www.nature.com/articles/s41586-021-03819-2) a révolutionné le domaine de la prédiction de la structure des protéines, gagnant une adoption et une reconnaissance généralisées dans les milieux universitaires, biotechnologiques et pharmaceutiques :

<div align="center">
    <img src="https://spectrum.ieee.org/media-library/two-examples-of-protein-targets-in-the-free-modelling-category-in-green-is-the-experimental-result-in-blue-is-the-computationa.gif?id=25559695&width=2400&height=1358" alt="Prédictions d'AlphaFold" width="800">
</div>


Des chercheurs d'[EvolutionaryScale](https://www.evolutionaryscale.ai/blog/esm3-release) ont utilisé leur modèle de langage protéique ESM3 pour concevoir une nouvelle protéine fluorescente assez distincte des protéines fluorescentes présentes dans la nature :

<div align="center">
    <img src="https://cdn.prod.website-files.com/6606dc3fd5f6645318003e20/667a5bb780d0ada7dc37d1c0_image%20(1).png" alt="ESM3" width="400">
</div>


### Écologie et conservation

[Rainforest Connection (RFCx)](https://rfcx.org/) est un projet innovant qui place des smartphones modifiés dans les arbres, enregistre des données audio, puis utilise des modèles pour identifier les différentes espèces présentes dans la zone. Cela permet de surveiller la biodiversité et de détecter les activités illégales comme l'exploitation forestière en temps réel.

<div align="center">
    <img src="https://www.huawei.com/~/media/CORPORATE/Images/case-studies/case1/photo-grid.jpg" alt="Configuration de Rainforest connection" width="600">
</div>

<div align="center">
    <img src="https://cdn.ttgtmedia.com/visuals/LeMagIT/Forest1.png" alt="Audio de Rainforest connection" width="600">
</div>



[Project CETI](https://www.projectceti.org/) (Cetacean Translation Initiative) utilise l'IA pour analyser et décoder les sons des baleines afin de comprendre leur communication et leur comportement.


<div align="center">
    <img src="https://i0.wp.com/www.josephdelpreto.com/wp-content/uploads/2023/09/Project-CETI_s-Approach-_-Illustration-%C2%A9-2023-Alex-Boersma.jpg?resize=1024%2C912" alt="Baleine CETI" width="600">
</div>


Un concours Zindi appelé [Turtle Recall](https://zindi.africa/competitions/turtle-recall-conservation-challenge) a mis les utilisateurs au défi de construire un modèle capable d'identifier les tortues marines individuelles à partir des motifs d'écailles sur leur tête, ce qui pourrait contribuer à améliorer les efforts de conservation des tortues marines :

<div align="center">
    <img src="https://lh3.googleusercontent.com/9J6DZgiuGyYr3N1DoJBmZMVpBkTlGOq19QUws7G2fbFcuHeIJKQ3plFh-R2xkxB1OpVaqZhcglM6hWWl5x7PuuxbtnDlIWlCgoCr0LGVM4S-loaj_Jc=w1232-rw" alt="Rappel des tortues" width="600">
</div>


**Sondage** : Levez la main, lequel de ces sous-domaines de l'IA pour la biologie trouvez-vous le plus intéressant ?
1. 🏥 **Diagnostics médicaux** 🏥
2. 💊 **Pharmacie et développement de médicaments** 💊
3. 🧬 **Biologie moléculaire** 🧬
4. 🌳 **Écologie et conservation** 🌳
5. **Autre** (lequel ? :)

**Question** : quelles autres initiatives intéressantes dans le domaine de l'IA en biologie connaissez-vous ?


### Lectures complémentaires

Ces exemples ne sont pas exhaustifs et visent simplement à vous donner un aperçu de certaines applications actuelles de l'IA en biologie. Si vous souhaitez en savoir plus sur ce domaine, voici quelques ressources intéressantes à lire :

- Un article de revue de *Nature Communications* de 2022 intitulé ["Current progress and open challenges for applying deep learning across the biosciences"](https://www.nature.com/articles/s41467-022-29268-7)
- Un article de revue un peu plus ancien (2018) intitulé [“Opportunities and obstacles for deep learning in biology and medicine”](https://royalsocietypublishing.org/doi/10.1098/rsif.2017.0387). Celui-ci a été cité plus de 2 000 fois !
- ["Deep Learning for the Life Sciences"](https://www.oreilly.com/library/view/deep-learning-for/9781492039822/), un livre d'O'Reilly de 2019, qui fournit des informations pratiques et des applications du deep learning en génomique, en chimie et en bio-informatique.


## 2. Introduction à l'ADN

    


Nous espérons que cette introduction vous a suffisamment enthousiasmé pour les applications de l'apprentissage profond en biologie ! Pour le reste de cette séance pratique, nous allons nous plonger nous-mêmes dans le travail d'IA appliqué à la biologie, en nous concentrant sur le sujet de l'ADN.


<div align="center">
    <img src="https://i.pinimg.com/originals/c7/90/76/c79076215950e968828f663e1b69fe67.gif" alt="DNA gif" width="200">
</div>

    


**L'ADN est la molécule de l'hérédité, la base de toute vie telle que nous la connaissons.**

Sa structure a été découverte pour la première fois en 1953, marquant un moment crucial dans les sciences biologiques. La première ébauche du génome humain a été publiée en 2001, jetant les bases de la génomique moderne.

Mais ces dates sont assez récentes, et bien que nous connaissions maintenant une partie du "quoi" du génome, nous sommes très loin de connaître le "comment" de son fonctionnement réel.

Par exemple, nous savons que l'ADN est composé de 4 "lettres" différentes, à savoir A (adénine), C (cytosine), G (guanine) et T (thymine) et que le génome humain est composé de 3,2 milliards de lettres qui sont réparties sur 23 paires de chromosomes. L'ADN est compacté de différentes manières afin de tenir dans le noyau de la cellule :

<div align="center">
    <img src="https://miro.medium.com/v2/resize:fit:1400/1*EUKrGpPzUAwp2sOOPZOcqA.jpeg" alt="DNA text" width="400">
</div>

Si nous devions ouvrir le "livre" du génome humain, nous verrions quelque chose comme ceci :

<div align="center">
    <img src="https://cms.wellcome.org/sites/default/files/styles/image_full_hi/public/WI_C0035768_GenomeEditing_20150902_News_600x600.jpg?itok=FCedpedU" alt="DNA text" width="400">
</div>

**Ceci peut être très difficile à interpréter**. Et il y a beaucoup d'ADN à interpréter - assez pour remplir une bibliothèque s'il était imprimé dans des volumes de livres :

<div align="center">
    <img src="https://live.staticflickr.com/3265/2569126918_b68047a65b_b.jpg" alt="DNA bookcase" width="400">
</div>

Parmi les principales questions ouvertes que nous nous posons sur l'ADN, citons :
- Que font les 3,2 milliards de lettres du génome humain ? Sont-elles toutes biologiquement fonctionnelles ?
- Comment chaque cellule du corps humain peut-elle avoir exactement le même génome, mais avoir des fonctions très différentes ? Par exemple, pensez à la différence entre un neurone et une cellule musculaire.
- Nous savons que seulement 2 % environ de l'ADN du génome code pour des protéines, à quoi servent les 98 % restants ?
- Comment la variation génétique conduit-elle à la maladie ou aux différences que nous observons entre les individus ?

Pour résumer :

***Le génome humain peut être considéré comme un très long livre dont les mots sont composés des lettres A, T, C et G. Les modèles de deep learning sont très prometteurs pour la compréhension du génome en raison de leur capacité à saisir le signal dans des données volumineuses, complexes et potentiellement bruyantes.***
    


## Modèles de langage ADN
    



## Modèles de langage ADN

**Les modèles de langage ADN (LMs)** sont très similaires aux modèles de langage de grande taille que vous connaissez peut-être, tels que ChatGPT, Gemini, Claude, etc. Au lieu d'être entraînés sur de grandes quantités de texte en langage naturel, les LMs d'ADN sont entraînés sur de grandes quantités de séquences d'ADN.

De nombreux LLMs et LMs d'ADN sont entraînés en **masquant** de manière aléatoire certains jetons dans le texte, puis en demandant au modèle de prédire ce qu'est le jeton :

<div align="center">
<a href="https://ibb.co/M9d9GM0"><img src="https://i.ibb.co/25J5sg4/Screenshot-2024-07-23-at-22-18-57.png" alt="Screenshot-2024-07-23-at-22-18-57" width="800" border="0"></a>
</div>

De la même manière que les LLMs acquièrent une compréhension du langage qui peut ensuite être utile pour de nombreuses tâches en aval, les modèles de langage ADN peuvent capturer les schémas et les structures complexes au sein de l'ADN, ce qui les rend précieux pour diverses tâches génomiques en aval telles que l'analyse des mutations et la compréhension des éléments régulateurs :


## Quelques modèles de langage ADN populaires

Tout comme les LLMs ont connu un succès massif, de plus en plus de personnes s'intéressent à l'entraînement de modèles de langage ADN. Parmi les plus célèbres, citons :

- [DNABERT](https://academic.oup.com/bioinformatics/article/37/15/2112/6128680) (2021) - DNABERT adapte le modèle BERT, qui a connu un grand succès en PNL (NLP), pour comprendre les séquences d'ADN.
- [HyenaDNA](https://arxiv.org/abs/2306.15794) (2023) - Spécialisé dans les longues séquences d'ADN, avec des contextes allant jusqu'à 1 million de jetons au niveau du nucléotide unique.
- [Nucleotide Transformer](https://www.biorxiv.org/content/10.1101/2023.01.11.523679v1) (2023) - une architecture basée sur un transformateur spécifiquement conçue pour les séquences de nucléotides.

Certaines personnes ont également affiné des LLMs en langage naturel existants sur des séquences d'ADN, par exemple [Mistral-DNA](https://github.com/raphaelmourad/Mistral-DNA).

Pour ce TP, nous utiliserons le modèle Nucleotide Transformer
(NT) car il est assez performant, populaire et facilement disponible sur la [plateforme Hugging Face 🤗](https://huggingface.co/).


## Le modèle Nucleotide Transformer (NT)
    


Nucleotide Transformer a été entraîné sur 3 202 génomes humains divers, ainsi que sur 850 génomes provenant d'un large éventail d'espèces. Le modèle génère des représentations transférables et contextuelles de séquences d'ADN.

Voici quelques détails supplémentaires sur le modèle :
- NT est une architecture de transformateur à encodeur uniquement, formée à l'aide de l'approche BERT (masquage de parties de séquences d'ADN). Les séquences d'ADN ont été tokenisées en 6-mers.
- Il s'agit d'un modèle non supervisé, mais ses représentations seules égalent ou surpassent les méthodes spécialisées sur 11 tâches de prédiction sur 18, telles que la prédiction de la présence de certains éléments régulateurs connus sous le nom de promoteurs et d'amplificateurs dans un fragment d'ADN donné.
- Les données d'entraînement pour la version `nucleotide-transformer-v2-50m-multi-species` de NT ont été entraînées sur un total de **174 milliards de nucléotides**, soit environ **29 milliards de jetons (tokens)**. Voici les statistiques par groupe d'organismes :

| Classe                | Nombre d'espèces | Nombre de nucléotides (milliards) |
| ---------------------| -------------------| --------------------------|
| Bactéries            | 667                | 17,1                      |
| Champignons           | 46                 | 2,3                       |
| Invertébrés          | 39                 | 20,8                      |
| Protozoaires          | 10                 | 0,5                       |
| Vertébrés mammifères | 31                 | 69,8                      |
| Autres vertébrés      | 57                 | 63,4                      |

Il existe d'autres versions plus volumineuses du modèle disponibles sur Hugging Face [ici](https://huggingface.co/collections/InstaDeepAI/nucleotide-transformer-65099cdde13ff96230f2e592). Nous utilisons ici un modèle relativement petit pour des raisons de vitesse.
    


## Exploration des plongements d'ADN dans différentes espèces
    


Chargeons le modèle Nucleotide Transformer (NT) depuis HuggingFace et commençons à l'utiliser !

Nous avons déjà chargé le tokenizer et le modèle dans la première cellule du notebook. Ce sont ces deux objets :
    


In [ ]:
type(tokenizer)

In [ ]:
type(language_model)

    
Voyons ce que le modèle a appris sur l'ADN de différentes espèces. Nous pouvons prendre des séquences d'ADN aléatoires provenant de différentes espèces, utiliser le modèle pour extraire une représentation **embedding** de l'ADN, et voir si les espèces similaires ont tendance à avoir un ADN avec des représentations similaires.

Créons une liste d'espèces qui nous intéressent :
    


In [ ]:
organismes = [
    'Homo sapiens',  # Humain
    'Pan troglodytes',  # Chimpanzé
    'Pan paniscus',     # Bonobo
    'Gorilla gorilla',  # Gorille
    'Tursiops truncatus',  # Dauphin (à gros nez)
    'Hydrochoerus hydrochaeris',  # Capybara
    'Escherichia coli',  # Bactérie E. coli
    'Lactobacillus acidophilus',  # Bactérie probiotique commune
    'Salmonella enterica',  # Pathogène d'origine alimentaire courant
    'Pseudomonas aeruginosa',  # Bactérie présente dans le sol et l'eau
]

print(len(organismes))

On peut utiliser la bibliothèque Python `Entrez` pour effectuer une recherche dans la base de données pour une chaîne d'ADN aléatoire pour un organisme donné :


In [ ]:
organism = 'Homo sapiens'
gene = 'BRCA1'  # Nom de gène exemple.

Entrez.email = "your.email@example.com"  # Requis. Peut être un espace réservé.

# Recherche des enregistrements d'ADN pour l'organisme.
query = f'({gene}[Gene Name]) AND {organism}[Organism] AND "RefSeq"[filter]'
handle = Entrez.esearch(db='nucleotide', term=query, retmax=10)
record = Entrez.read(handle)
handle.close()

# Récupérer un enregistrement aléatoire.
np.random.seed(42)
random_record_id = np.random.choice(record['IdList'])

# Lire l'ADN.
handle = Entrez.efetch(
    db='nucleotide', id=random_record_id, rettype='fasta', retmode='text')
seq_record = SeqIO.read(handle, 'fasta')

print(seq_record)

On peut facilement extraire la chaîne d'ADN de cette entrée :
    


In [ ]:
dna_sequence = str(seq_record.seq)
dna_sequence

Le modèle NT peut être utilisé pour calculer une représentation numérique de la signification de cette séquence d'ADN. Nous allons simplement passer la séquence d'ADN en entrée, la tokeniser, et extraire les activations de la dernière couche cachée du modèle :


In [ ]:
# Tokenisation de la séquence d'ADN.
max_length = tokenizer.model_max_length

token_ids = tokenizer.batch_encode_plus(
  [dna_sequence], return_tensors='pt',
  padding='max_length', max_length=max_length,
  truncation=True)['input_ids']

token_ids

Vous pouvez voir que la séquence d'ADN a été tokenisée et complétée (padded) avec le token `1` jusqu'à la longueur maximale de 2048 :


In [ ]:
len(token_ids[0])

In [ ]:
token_ids[0][-50:]  # Afficher les 50 derniers jetons.

Voyons maintenant la sortie du modèle étant donné cette entrée d'ADN :
    


In [ ]:
masque_attention = token_ids != tokenizer.pad_token_id

torch_outs = language_model(
  token_ids,
  attention_mask=masque_attention,
  encoder_attention_mask=masque_attention,
  output_hidden_states=True,
)

On peut voir que 13 sorties d'états cachés différentes sont présentes, représentant les sorties de 13 couches différentes dans le modèle :


In [ ]:
len(torch_outs['hidden_states'])

Prenons le dernier état caché à l'index `-1` car il est susceptible d'être informatif (mais peut-être pas de manière optimale pour chaque tâche - nous pourrions essayer l'avant-dernier état caché, ou le troisième à partir de la fin, etc.) :
    


In [ ]:
embeddings = torch_outs['hidden_states'][-1].detach()
embeddings = np.squeeze(embeddings.numpy())
embeddings.shape

On peut voir que le plongement est une matrice de forme 2048 par 512. C'est une grande matrice numpy !
    


In [ ]:
embeddings

Au sein de cette large matrice de nombres, le modèle a capturé un certain sens de la signification de la chaîne d'ADN.

Rappelons que notre chaîne d'ADN avait une longueur de 2048 – cela signifie que chaque position a son propre vecteur de plongement (embeddings) de longueur 512 qui est contextuel.

Nous pourrions visualiser cette matrice de nombres, mais cela en soi ne serait pas très significatif :

    


In [ ]:
plt.imshow(embeddings)
plt.show()

Une façon courante de résumer un plongement comme celui-ci est de calculer le **plongement moyen** : nous pouvons prendre la moyenne sur l’axe spatial et obtenir un plongement de longueur 512 qui représente la séquence d’ADN entière.


In [ ]:
# Déplacer l'axe pour que le broadcasting fonctionne.
masque_attention = np.moveaxis(masque_attention.numpy(), 0, -1)

# Calcul du plongement (embeddings) moyen.
mean_embeddings = np.sum(
    masque_attention * embeddings, axis=0) / np.sum(masque_attention)

plt.plot(mean_embeddings, color='grey')
plt.show()

**C'est pratique de pouvoir condenser une séquence d'ADN en 512 nombres comme ceci, mais c'est encore difficile de vraiment savoir ce que ces nombres signifient.**

La valeur des plongements (embeddings) de séquences se révèle vraiment lorsque vous les **comparez les uns aux autres**. Donc, écrivons la fonction `fetch_random_dna` qui va extraire des séquences d'ADN aléatoires pour une espèce donnée :
    


In [ ]:
def fetch_random_dna(
  organism: str,
  min_length: int,
  max_length: int,
  num_sequences: int,
  max_attempts: int = 50) -> list[str]:
  """Récupère un certain nombre de séquences d'ADN d'une longueur donnée pour une espèce."""
  # Recherche des entrées nucléotidiques pour l'organisme comme nous l'avons fait précédemment.
  handle = Entrez.esearch(
      db='nucleotide', term=f'{organism}[Organism]', retmax=max_attempts)
  record = Entrez.read(handle)
  handle.close()
  if not record['IdList']:
    return []

  # Nous pouvons collecter nos séquences d'ADN dans cette liste.
  sequences = []  # (séquences)
  attempts = 0  # (tentatives)

  while len(sequences) < num_sequences and attempts < max_attempts:
    random_record_id = random.choice(record['IdList'])
    handle = Entrez.efetch(
        db='nucleotide', id=random_record_id, rettype='fasta', retmode='text')
    try:
      seq_record = SeqIO.read(handle, 'fasta')
      handle.close()

      if len(seq_record.seq) >= min_length:
        seq = str(seq_record.seq) # (seq)
        if len(seq) > max_length:
          seq = seq[:max_length]
        sequences.append(seq)

    # Gère le cas où aucun enregistrement FASTA valide n'a été trouvé.
    except ValueError:
      handle.close()
    attempts += 1

  return sequences

Utilisons cela pour récupérer une courte séquence d'ADN pour l'homme :
    


In [ ]:
fetch_random_dna(
    organism='Homo sapiens', min_length=10, max_length=50, num_sequences=1)

Maintenant, on peut facilement l'utiliser pour récupérer des séquences d'ADN pour les organismes dans notre liste ci-dessus. Le code pour faire cela ressemblerait à ceci :
    


In [ ]:
NUM_SEQUENCES = 1
MIN_LENGTH = 100
MAX_LENGTH = 1000
random.seed(42)

# (first_organisms) : premiers organismes
first_organisms = organismes[0:3]

print(f'Fetching {NUM_SEQUENCES} random DNA sequences of min length '
      f'{MIN_LENGTH} and max length {MAX_LENGTH} for {len(first_organisms)} '
      'organisms...\n')

dna_sequences_small = []
# (organism_labels) : étiquettes des organismes
organism_labels = []

# (organism) : organisme
for organism in tqdm.tqdm(first_organisms, desc='Organisms'):
  print(organism, flush=True)
  # (sequences) : séquences
  sequences = fetch_random_dna(
      organism, min_length=MIN_LENGTH, max_length=MAX_LENGTH,
      num_sequences=NUM_SEQUENCES)
  dna_sequences_small += sequences
  organism_labels += [organism] * len(sequences)

# (sequence) : séquence
dna_sequences_small = pd.DataFrame({'sequence': dna_sequences_small, 'organism': organism_labels})

Mais cela prend quelques minutes à s'exécuter si nous voulons par exemple 20 séquences d'ADN pour chacun des 10 organismes, donc par souci de rapidité, nous avons pré-récupéré certaines séquences pour plus de commodité :
    


In [ ]:
dna_sequences

Nous pouvons nous appuyer sur notre code précédent pour extraire les plongements moyens des séquences d'ADN :
    



In [ ]:
def _compute_mean_sequence_embeddings(
  dna_sequences: list[str],
  tokenizer: AutoTokenizer,
  model: AutoModelForMaskedLM):

  max_length = tokenizer.model_max_length
  tokens_ids = tokenizer.batch_encode_plus(
    dna_sequences, return_tensors="pt", padding="max_length",
    max_length=max_length)["input_ids"]

  # Calcul des plongements.
  attention_mask = tokens_ids != tokenizer.pad_token_id

  # Déplacement du modèle et des tenseurs vers le GPU.
  model = model.to('cuda')
  tokens_ids = tokens_ids.to('cuda')
  attention_mask = attention_mask.to('cuda')

  # Par défaut, PyTorch conserve le graphe de calcul pour la passe arrière, mais cela
  # remplit la RAM et nous n'en avons pas besoin, nous le désactivons donc avec torch.no_grad().
  with torch.no_grad():
    torch_outs = model(
      tokens_ids,
      attention_mask=attention_mask,
      encoder_attention_mask=attention_mask,
      output_hidden_states=True,
    )

  # Calcul des plongements de séquences.
  embeddings = torch_outs['hidden_states'][-1].detach().cpu()

  # Ajout d'une dimension de plongement.
  attention_mask_cpu = torch.unsqueeze(attention_mask.cpu(), dim=-1)

  # Calcul des plongements moyens par séquence
  mean_sequence_embeddings = torch.sum(
    attention_mask_cpu * embeddings, axis=-2) / torch.sum(attention_mask_cpu, axis=1)

  return mean_sequence_embeddings.numpy()


def compute_mean_sequence_embeddings(
    dna_sequences: list[str],
    tokenizer: AutoTokenizer,
    model: AutoModelForMaskedLM,
    batch_size: int = 20) -> np.ndarray:
  """Calcule les plongements moyens de séquences pour une liste de chaînes d'ADN."""
  all_mean_embeddings = []

  # (batch_sequences: lots de séquences, batch_mean_embeddings: plongements moyens des lots)
  for i in tqdm.tqdm(range(0, len(dna_sequences), batch_size)):
    batch_sequences = dna_sequences[i:i+batch_size]
    batch_mean_embeddings = _compute_mean_sequence_embeddings(
        batch_sequences, tokenizer, model)
    all_mean_embeddings.extend(batch_mean_embeddings)

  return np.vstack(all_mean_embeddings)

In [ ]:
embeddings = compute_mean_sequence_embeddings(
    dna_sequences['sequence'], tokenizer, language_model, batch_size=7)

Cela nous donne un encodage de longueur 512 pour chacune des 200 chaînes d'ADN :


In [ ]:
embeddings.shape

Ce serait formidable de pouvoir visualiser ces données. Mais comme les humains ne peuvent pas vraiment visualiser des choses dans un espace à 512 dimensions, utilisons d'abord une technique de réduction de dimensionnalité telle que tSNE pour projeter les données sur 2 dimensions. Cela donne 2 nombres pour chaque séquence d'ADN originale qui capturent encore une certaine notion de sens dans l'ADN :
    


In [ ]:
tsne = TSNE(n_components=2, learning_rate='auto', random_state=0)
embeddings_tsne = tsne.fit_transform(embeddings)

# (embeddings_tsne_df) : DataFrame des embeddings après tSNE
embeddings_tsne_df = pd.DataFrame(
    embeddings_tsne, columns=['first_dim', 'second_dim'])

# (organism) : Organisme
embeddings_tsne_df['organism'] = dna_sequences['organism']
embeddings_tsne_df

# On peut étiqueter chaque token avec son espèce ou une étiquette plus générale, comme "animal" ou "plante".
    # Cela nous permet de voir si des séquences similaires provenant d'espèces similaires se regroupent.
    # On peut ensuite redessiner le graphique et le colorer par étiquette :


In [ ]:
labels = {
    'Homo sapiens': 'animal',
    'Pan paniscus': 'animal',
    'Pan troglodytes': 'animal',
    'Tursiops truncatus': 'animal',
    'Hydrochoerus hydrochaeris': 'animal',
    'Escherichia coli': 'bactérie',
    'Pseudomonas aeruginosa': 'bactérie',
    'Lactobacillus acidophilus': 'bactérie',
    'Salmonella enterica': 'bactérie',
    }

embeddings_tsne_df['label'] = embeddings_tsne_df['organism'].map(labels)

ax = sns.scatterplot(data=embeddings_tsne_df,
                x='first_dim',
                y='second_dim',
                hue='label', color=None,
                s=200, alpha=0.7, palette='Set2')

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()

Il semble que, bien que les séquences animales et bactériennes aient tendance à occuper des parties quelque peu différentes de l'espace d'incorporation, il existe certainement beaucoup de chevauchements, ce qui suggère que les génomes de l'arbre de vie partagent de nombreuses similitudes !

**Question **: Est-ce ce à quoi vous vous attendiez ?

Si vous souhaitez en savoir plus sur les génomes animaux et bactériens, voici quelques faits amusants :
- ***Similitudes*** :
  - **Code génétique de base** : les génomes animaux et bactériens utilisent le même code génétique, avec des séquences d'ADN composées des quatre mêmes nucléotides : adénine (A), cytosine (C), guanine (G) et thymine (T).
  - **Gènes conservés** : De nombreux gènes fondamentaux impliqués dans des processus essentiels, tels que la réplication de l'ADN, la transcription et la traduction, sont conservés chez les animaux et les bactéries.
- ***Différences*** :
  - **Taille du génome** : les génomes animaux sont généralement beaucoup plus gros (les humains ont 3,2 milliards de bases d'ADN) tandis que les génomes bactériens sont plus petits (de quelques centaines de milliers à quelques millions).
  - **Chromosomes** : les animaux ont plusieurs chromosomes linéaires, tandis que les bactéries ont généralement un seul chromosome circulaire.
  - **Densité des gènes** : les génomes bactériens sont plus denses en gènes, tandis que les génomes animaux sont plus rares et comportent plus d'éléments régulateurs.
  - **Structure des gènes** : les gènes animaux contiennent souvent des introns (régions non codantes au sein des gènes) contrairement aux gènes bactériens en général.

**Question :** Lesquelles de ces différences pourraient être capturées par les embeddings mentionnés ci-dessus ?


**Tâche** :
- Essayez de saisir des séquences d'ADN pour d'autres espèces, par exemple des espèces végétales. Voici quelques noms scientifiques de plantes :
```python
 plants = [
  'Oryza sativa',  # Riz
  'Vitis vinifera',  # Raisin
  'Rosa chinensis',  # Rose
  'Musa acuminata',  # Banane
  'Solanum lycopersicum', # Tomate
 ]
 ```
- Dans quelle mesure le nuage de points est-il sensible aux changements de germe aléatoire ou aux autres paramètres ? Essayez une technique de réduction de dimensionnalité différente telle que UMAP au lieu de tSNE. Le nuage de points est-il différent ?
    

## 3. Réglage fin d'un modèle de langage ADN



Dans cette dernière section, nous allons adapter notre modèle de langage ADN à une nouvelle tâche par **réglage fin** (fine-tuning).

### Qu'est-ce que le réglage fin ?
Le réglage fin est le processus qui consiste à prendre un **modèle pré-entraîné** et à y apporter de légères modifications afin qu'il puisse effectuer une nouvelle tâche spécialisée.

Au lieu d'entraîner un modèle à partir de zéro, ce qui peut prendre beaucoup de temps et nécessiter beaucoup de données, nous commençons par un modèle qui comprend déjà certains concepts généraux. Nous l'entraînons ensuite sur un ensemble de données plus petit et spécifique à la tâche.

Le **pré-entraînement suivi d'un réglage fin** est un modèle général dans le domaine du ML. Voici quelques exemples tirés du langage naturel et du langage ADN :


<a href="https://ibb.co/tpd50VH"><img src="https://i.ibb.co/CKrFjRw/ML-for-bio-06.png" alt="NLP vs DNA" border="0" width="400"></a>


### Le problème biologique

--> **Nous allons entraîner un modèle pour prédire si une chaîne donnée de 200 bases d'ADN va se lier à un facteur de transcription donné**.

Les facteurs de transcription (FT) sont des protéines spéciales qui se lient à l'ADN et jouent un rôle crucial dans l'activation ou la désactivation des gènes. Ils sont essentiels car ils contrôlent l'expression des gènes, qui à son tour affecte le fonctionnement, le développement et la réponse des cellules à leur environnement. Par exemple, ils peuvent déterminer si une cellule devient une cellule musculaire, un neurone ou une cellule cutanée.

Voici une image d'un facteur de transcription (en violet) se liant à une certaine région de l'ADN (surlignée en jaune) :

<div align="center">
    <img src="https://www.nichd.nih.gov/sites/default/files/2022-05/TranscriptionFactor-400px.jpg" alt="DNA TF binding" width="400">
</div>






Chaque facteur de transcription a une certaine **préférence de liaison** - il préfère se lier à une séquence spécifique de bases d'ADN et pas à d'autres. Cela est dû au fait que les formes 3D du FT et de la région de l'ADN peuvent bien s'assembler ou non.

L'homme possède plus de 1 000 facteurs de transcription. Nous allons nous intéresser à un facteur de transcription spécifique appelé CTCF, qui a tendance à se lier à des séquences similaires à CCACCAGGGGGCGC (avec une certaine variation possible à certaines positions).

Voici le problème de prédiction en termes visuels :


- Pour une chaîne spécifique de 200 paires de bases d'ADN, nous voulons prédire la probabilité qu'un **facteur de transcription** donné se lie dans cette région.


## Le jeu de données
    


Voici à quoi ressemble le jeu de données que nous allons utiliser :

<a href="https://ibb.co/1ZbY3SF"><img src="https://i.ibb.co/PxtsRSq/ML-for-bio-05.png" alt="dataset description" border="0"></a>

La tâche est une **tâche de classification binaire** – étant donné 200 bases d'ADN, nous prédisons s'il se liera à un facteur de transcription spécifique appelé CTCF. Le CTCF est en fait un facteur de transcription particulièrement intéressant, car il est impliqué dans l’**architecture du génome**, c'est-à-dire le repliement 3D élaboré du génome en compartiments spécifiques.

Le problème est inspiré de l'une des tâches d'évaluation de ce récent [pré-impression d'article de 2024](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10925287/), qui a tiré le jeu de données de cet [article d'interprétation de la génomique de 2023](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10169356/)
    


#### Chargement du jeu de données.
    


Le jeu de données d'entraînement a déjà été construit pour vous :
- Nous avons 20 000 exemples d'entraînement.
- Chacun est un encodage (embedding) moyen de l'ADN qui a été extrait à l'aide du modèle de langage NT.
- La dernière colonne du dataframe est l'étiquette, indiquant si l'ADN se lie ou non à la protéine CTCF.


In [ ]:
train_df

En général, on observe que les 2 classes sont assez équilibrées (représentées de manière égale) dans le jeu de données d'entraînement, ce qui signifie que nous n'aurons pas besoin de faire de rééquilibrage ici :
    


In [ ]:
train_df['label'].value_counts()

Si cela vous intéresse, vous pouvez consulter le code qui a permis de générer ce jeu de données, mais vous n'avez pas besoin de l'exécuter ici.


#### [Vous n'avez pas besoin d'exécuter ceci] Code de création du jeu de données.

```python
# 1. Charger le fichier h5 contenant les jeux de données CTCF.
file_path = os.path.join(ROOT_DIR, 'CTCF_200.h5')

with h5py.File(file_path, 'r') as h5file:
  print("Keys: %s" % list(h5file.keys()))

  # Accéder à chaque jeu de données et le convertir en tableaux numpy.
  x_train = h5file['x_train'][()] # données d'entraînement
  y_train = h5file['y_train'][()] # étiquettes d'entraînement
  x_valid = h5file['x_valid'][()] # données de validation
  y_valid = h5file['y_valid'][()] # étiquettes de validation
  x_test = h5file['x_test'][()] # données de test
  y_test = h5file['y_test'][()] # étiquettes de test

# Chaque séquence d'ADN est encodée en one-hot. Visualiser le premier exemple d'entraînement :
fig, ax = plt.subplots(figsize=(12, 12))
plt.imshow(x_train[0, :, :])
plt.show()

# 2. Étant donné que notre modèle de langage d'ADN prend en réalité des lettres en entrée, nous pouvons annuler
# l'encodage one-hot avec une fonction :

def one_hot_to_dna_batch(one_hot_encoded_batch: np.ndarray):
  """
  Convertir un lot de séquences d'ADN encodées en one-hot en chaînes de séquences d'ADN.

  Args:
    one_hot_encoded_batch (numpy.ndarray): Un tableau numpy 3D avec des séquences d'ADN
      encodées en one-hot. La forme doit être (nombre_de_séquences, longueur_de_la_séquence, 4).

  Returns:
    list: Une liste de séquences d'ADN.
  """
  # Définir une correspondance entre l'encodage one-hot et les nucléotides.
  one_hot_mapping = {
      (1, 0, 0, 0): 'A',
      (0, 1, 0, 0): 'C',
      (0, 0, 1, 0): 'G',
      (0, 0, 0, 1): 'T',
  }

  dna_sequences = []

  for one_hot_encoded in one_hot_encoded_batch:
    dna_sequence = []
    for one_hot in one_hot_encoded:
      one_hot_tuple = tuple(one_hot)
      dna_sequence.append(one_hot_mapping[one_hot_tuple])

    dna_sequences.append(''.join(dna_sequence))

  return dna_sequences

NUM_TRAIN_EXAMPLES = 20_000
NUM_VALID_EXAMPLES = 5_000

# exemples d'entraînement (x_train), étiquettes d'entraînement (y_train), exemples de validation (x_valid), étiquettes de validation (y_valid)
x_train = one_hot_to_dna_batch(
    np.moveaxis(x_train, 1, -1)[0:NUM_TRAIN_EXAMPLES])
y_train = y_train[0:NUM_TRAIN_EXAMPLES]

x_valid = one_hot_to_dna_batch(
    np.moveaxis(x_valid, 1, -1)[0:NUM_VALID_EXAMPLES])
y_valid = y_valid[0:NUM_VALID_EXAMPLES]

# Jeter un coup d'œil aux exemples d'entraînement et aux étiquettes :
print(x_train[0:5])
print(y_train[0:5])

# 3. Calculer les plongements moyens du modèle de langage d'ADN des séquences.
train_embeddings = compute_mean_sequence_embeddings(
    x_train, tokenizer, language_model)
train_df = pd.DataFrame(train_embeddings)
train_df['label'] = y_train[:, 0]

valid_embeddings = compute_mean_sequence_embeddings(
    x_valid, tokenizer, language_model)
valid_df = pd.DataFrame(valid_embeddings)
valid_df['label'] = y_valid[:, 0]
```
    


## Convertir les données en un jeu de données TensorFlow
    


Nous devrons convertir ces dataframes en un jeu de données TensorFlow sur lequel nous pourrons facilement itérer lors de l'entraînement du modèle :
    


In [ ]:
import tensorflow as tf
import numpy as np

def convert_to_tfds(df: pd.DataFrame, batch_size: int=32,
                    is_training: bool=False):
    """Convertit les plongements et les étiquettes en un jeu de données TensorFlow.""" # embeddings: plongements, labels: étiquettes
    embeddings = np.array(df.iloc[:, :-1])
    labels = np.array(df.iloc[:, -1])[:, None]

    ds = tf.data.Dataset.from_tensor_slices(
        {'embeddings': embeddings, 'labels': labels})

    if is_training:
      ds = ds.shuffle(buffer_size=len(df)).repeat()

    ds = ds.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    return iter(ds)

BATCH_SIZE = 32

train_ds = convert_to_tfds(
    train_df, batch_size=BATCH_SIZE, is_training=True)
valid_ds = convert_to_tfds(
    valid_df, batch_size=BATCH_SIZE, is_training=False)

Jetons un coup d'œil à un lot de données d'entraînement :
    



In [ ]:
batch = next(train_ds)
batch

Le jeu de données est prêt pour l'entraînement du modèle !


## Réglage fin du modèle


Nous allons maintenant entraîner un modèle linéaire [flax](https://flax.readthedocs.io/en/latest/) simple sur les plongements d'ADN moyens.

Flax est assez similaire à de nombreux autres frameworks de deep learning (en particulier [Haiku](https://dm-haiku.readthedocs.io/en/latest/), si vous l'avez déjà rencontré). Dans notre configuration, notez que notre modèle n'est qu'un MLP (perceptron multicouche, qui est constitué de plusieurs couches linéaires avec quelques non-linéarités) - nous ne modifions pas (rétropropagation dans) le modèle linguistique d'ADN original.
    


In [ ]:
class Model(nn.Module):
  dim: int = 128

  @nn.compact
  def __call__(self, x):
    x = nn.Dense(self.dim * 2)(x)
    x = nn.gelu(x)
    x = nn.Dense(self.dim)(x)
    x = nn.gelu(x)
    x = nn.Dense(1)(x)
    return x

In [ ]:
mlp = Model()

### Boucle d'entraînement (Training loop)

Avec le modèle et les données mis en place, nous pouvons maintenant initialiser les paramètres de notre modèle, notre optimiseur, et écrire une fonction pour effectuer une seule étape d'entraînement (qui englobe une passe avant du modèle (model forward pass), un calcul de la perte (loss computation), un calcul du gradient (gradient computation) et une mise à jour des paramètres du modèle à l'aide des gradients) :
    



In [ ]:
LEARNING_RATE = 0.0001

init_rng = jax.random.PRNGKey(42)
variables = mlp.init(init_rng, batch['embeddings']) # batch: lot
params = variables['params']

optimiser = optax.adam(LEARNING_RATE)
opt_state = optimiser.init(params)

Vous pouvez vérifier les noms des couches dans notre réseau de neurones comme ceci :


In [ ]:
params.keys()

Et vérifiez que la forme de cette couche correspond à ce que vous attendez comme ceci :
    



In [ ]:
for layer_name in ['Dense_0', 'Dense_1', 'Dense_2']:
      print(params[layer_name]['kernel'].shape)

**Question** : Pouvez-vous déterminer d'où proviennent ces formes, étant donné notre code dans notre `class Model` ci-dessus ?


Nous pourrions déjà faire des prédictions en utilisant ces paramètres initialisés aléatoirement (seulement, les prédictions seront aléatoires) :
    


In [ ]:
preds = mlp.apply({'params': params}, batch['embeddings'])
nn.sigmoid(preds)

Définissons maintenant une fonction de perte que nous pouvons utiliser pour entraîner ces paramètres :
    


In [ ]:
def loss_fn(params, embeddings, labels):
  """Applique la fonction sigmoïde aux logits et calcule la perte d'entropie croisée binaire (binary cross-entropy loss)."""
  logits = mlp.apply({'params': params}, embeddings)
  loss = optax.sigmoid_binary_cross_entropy(
      logits=logits, labels=labels).mean()
  return loss

Calculons un exemple de perte :
     


In [ ]:
embeddings = jnp.array(batch['embeddings'])
labels = jnp.array(batch['labels']) # (labels: étiquettes)
loss_fn(params, embeddings, labels)

Donc, nous nous attendons à une perte autour de 0.6-0.7 pour des poids initialisés aléatoirement. Avec un peu de chance, avec l'entraînement du modèle, nous devrions voir des pertes plus faibles que cela à mesure que le modèle apprend le signal dans les données ! :)


Finalement, nous pouvons écrire une fonction étape d'entraînement :
    


In [ ]:
@jax.jit
def train_step(params, opt_state, embeddings, labels):
  """Une seule étape d'entraînement qui calcule les gradients et met à jour les paramètres du modèle."""
  loss, grads = jax.value_and_grad(loss_fn)(params, embeddings, labels)
  updates, opt_state = optimiser.update(grads, opt_state)
  params = optax.apply_updates(params, updates)
  return params, opt_state, loss

### Entraînons le modèle !


In [ ]:
NUM_EPOCHS = 5
NUM_TRAINING_STEPS = (len(train_df) // BATCH_SIZE) * NUM_EPOCHS
LEARNING_RATE = 0.001

# Réinitialise le modèle pour s'assurer qu'il démarre à zéro à chaque fois que la cellule est exécutée.
init_rng = jax.random.PRNGKey(42)
variables = mlp.init(init_rng, batch['embeddings'])
params = variables['params']

optimiser = optax.adam(LEARNING_RATE)
opt_state = optimiser.init(params)

# Conserve un enregistrement des pertes.
running_train_loss = None
running_train_losses = []
valid_losses = []

for epoch in tqdm.tqdm(range(NUM_EPOCHS)):

  # Boucle d'entraînement.
  for step in range(NUM_TRAINING_STEPS):
    batch = next(train_ds)
    embeddings = jnp.array(batch['embeddings'])
    labels = jnp.array(batch['labels'])
    params, opt_state, loss = train_step(params, opt_state, embeddings, labels)

    if running_train_loss is None:
      running_train_loss = loss.item()
    else:
      running_train_loss = 0.99 * running_train_loss + (1 - 0.99) * loss.item()
    running_train_losses.append(running_train_loss)

  # Boucle de validation.
  valid_ds = convert_to_tfds(valid_df, batch_size=BATCH_SIZE, is_training=False)
  for batch in valid_ds:
    embeddings = jnp.array(batch['embeddings'])
    labels = jnp.array(batch['labels'])
    loss = loss_fn(params, embeddings, labels)
    valid_losses.append(loss.item())

  valid_loss = np.mean(valid_losses)
  print(f'[Epoch {epoch}]: Valid loss (Perte de validation)={valid_loss:.3f}, '
        f'Train loss (Perte d\'entraînement)={running_train_loss:.3f}\n')

print('Entraînement terminé.')

🎉 🎉 **Et voilà, l'entraînement de base du modèle est terminé !** 🎉 🎉
    


## Vérification du modèle

Nous pouvons essayer d'inférer le modèle entraîné sur n'importe quelle nouvelle séquence d'ADN d'intérêt. Par exemple, comme nous savons grâce à des expériences biologiques que la protéine CTCF se lie aux séquences d'ADN contenant des motifs similaires à « CCACCAGGGGGCGC », le modèle devrait prédire une probabilité très élevée de liaison pour l'ADN contenant ces motifs.

Construisons la chaîne d'ADN de 200 bases et récupérons son incorporation :
    


In [ ]:
ctcf_motif_dna = 'CCACCAGGGGGCGC'*14 + 'AAAA'
print('Longueur de la chaîne d\'ADN remplie de motifs CTCF :', len(ctcf_motif_dna))

# (ctcf_motif_dna, ctcf_motif_embedding) -> (séquence_adn_motif_ctcf, incorporation_motif_ctcf)
ctcf_motif_embedding = compute_mean_sequence_embeddings(
    [ctcf_motif_dna], tokenizer, language_model)

Nous pouvons maintenant calculer la probabilité que l'ADN se lie au CTCF :
    


In [ ]:
jax.nn.sigmoid(mlp.apply({'params': params}, ctcf_motif_embedding))

Succès ! Cette probabilité est très proche de 1. Cela signifie que le modèle a appris à identifier une représentation de ce motif et à l'associer à la liaison de CTCF à l'ADN.

Inversement, les chaînes d'ADN aléatoires devraient avoir une faible probabilité de liaison avec CTCF :
    


In [ ]:
%%capture
random_dna_strings = [
    'ACGTACGT'*25,
    'CGGCCGCG'*25,
    'TCGATCGT'*25,
    'TTTTTTTT'*25,
]

probabilities = []

# (random_dna_string:chaîne d'ADN aléatoire)
for random_dna_string in random_dna_strings:
  # (random_dna_embedding:plongement d'ADN aléatoire)
  random_dna_embedding = compute_mean_sequence_embeddings(
    [random_dna_string], tokenizer, language_model)

  probabilities.append(
      jax.nn.sigmoid(mlp.apply({'params': params}, random_dna_embedding))[0])

In [ ]:
probabilities

Génial, celles-ci ressemblent toutes à des probabilités proches de zéro, ce à quoi nous nous attendions 😎.


## [Suivis Facultatifs]
    




1. **[Tracé des pertes]** Essayez de tracer la perte d'entraînement et la perte de validation au fil du temps. Qu'observez-vous ? Devrions-nous l'entraîner plus longtemps ? Y a-t-il un surajustement à l'ensemble d'entraînement ? Si oui, comment pourriez-vous améliorer la situation ?
 - **Indice** : essayez `plt.plot(train_losses, c='grey')`.
2. **[Métriques d'évaluation]** Jusqu'à présent, nous n'avons surveillé que les pertes pendant l'entraînement du modèle, mais celles-ci sont un peu difficiles à interpréter. Comment pourriez-vous implémenter et suivre une mesure de **précision** pendant l'entraînement ?
 - **Indice** : N'oubliez pas que si vous utilisez `jax.nn.sigmoid` sur les prédictions du modèle, cela donne la probabilité que la séquence d'ADN se lie à la protéine CTCF. Vous pouvez traiter toute probabilité supérieure à 0,5 comme une prédiction de '1', et toute probabilité inférieure à 0,5 comme une prédiction de '0'.
3. **[Réglage des hyperparamètres]** Essayez de faire varier le taux d'apprentissage, la taille du lot (batch size) et le nombre d'étapes d'entraînement. Comment ces changements affectent-ils la convergence et la performance finale du modèle ?
4. **[Augmentation des données]** Pouvez-vous penser à un moyen d'élargir (ou d'augmenter) l'ensemble d'entraînement ? Comment mesureriez-vous si cela est utile pour la performance du modèle ?
5. **[Architectures différentes]** Expérimentez avec différentes architectures de modèle, comme l'ajout de couches supplémentaires ou l'utilisation de différentes fonctions d'activation.
 - **Défi** : si vous vous sentez très aventureux, essayez d'implémenter un CNN capable d'apprendre directement à partir de séquences d'ADN (codées en one-hot) !


## Commentaires

N'hésitez pas à nous faire part de vos commentaires afin que nous puissions améliorer nos ateliers pratiques à l'avenir.
    

In [ ]:
# @title Générer un formulaire de commentaires (Exécuter la cellule)
from IPython.display import HTML

HTML(
    """
<iframe
	src="https://forms.gle/WUpRupqfhFtbLXtN6",
  width="80%"
	height="1200px" >
	Loading...
</iframe>
"""
)

<img src="https://baobab.deeplearningindaba.com/static/media/indaba-logo-dark.d5a6196d.png" width="50%" />